In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import gc
from captum.attr import *
import quantus
from torch.utils.data import DataLoader
import gc
import torchvision.transforms as transforms

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
#from rev2.cifar10.model_utils import resnet50, CIFAR10_RESNET50_CKPT_PATH

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, out_keys=None):
        out = {}
        x = self.conv1(x)
        out["c1"] = x
        x = self.bn1(x)
        out["bn1"] = x
        x = F.relu(x)
        out["r1"] = x

        x = self.layer1(x)
        out["l1"] = x
        x = self.layer2(x)
        out["l2"] = x
        x = self.layer3(x)
        out["l3"] = x
        x = self.layer4(x)
        out["l4"] = x

        x = F.avg_pool2d(x, 4)
        out["gvp"] = x
        x = x.view(x.size(0), -1)
        x = self.linear(x)
        out["fc"] = x

        if out_keys is None:
            return x
        res = {}
        for key in out_keys:
            res[key] = out[key]
        return res


def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])


def ResNet34():
    return ResNet(BasicBlock, [3,4,6,3])


def resnet50():
    return ResNet(Bottleneck, [3,4,6,3])


def ResNet101():
    return ResNet(Bottleneck, [3,4,23,3])


def ResNet152():
    return ResNet(Bottleneck, [3,8,36,3])


def test():
    net = ResNet18()
    y = net(torch.randn(1,3,32,32))
    print(y.size())

In [6]:
def load_cifar_model(path):
    model = resnet50()
    ckpt_dict = torch.load(path, lambda storage, loc: storage)
    model.load_state_dict(ckpt_dict)
    model.to('cuda')
    model.train(False)
    return model

In [7]:
modelpath = "/data/virtual environments/adv detection by robustness/adv_detection/Adaptive attacks/Models/CIFAR10/resnet50/cifar.ckpt"

In [8]:
normal_model = load_cifar_model(modelpath)
normal_model.to(device)
normal_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): Bottleneck(
      (

In [9]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                           download=True, transform=torchvision.transforms.ToTensor())
train_loader_cifar = DataLoader(trainset, shuffle=True, batch_size=10)

Files already downloaded and verified


# For given adversarial images and benign images, collect metrics of feature attribution sensitivity and model prediction sensitivity. Save in csv that will be used for inspecting detection performance. 

In [10]:
import pandas as pd

In [11]:
def make_noise(x_batch, y_batch, spread):
    new_x_batch = []
    for x in x_batch:
        x = x.data.cpu().numpy()
        stdev = spread * (np.max(x)-np.min(x))
        noise = np.random.normal(0, stdev, x.shape).astype(np.float32)
        x_plus_noise = x + noise
        x_plus_noise = np.clip(x_plus_noise, 0, 1)
        x_plus_noise = torch.from_numpy(x_plus_noise).cpu()
        new_x_batch.append(x_plus_noise)
    new_batch = torch.stack(new_x_batch).to(device)
    return new_batch

In [12]:
def compute_metrics_benign(adv_path, normal_model): 
    
    print("Computing metrics for {} for benign")
    
    npobj = np.load(adv_path)
    adaptive_image = npobj['b_images']
    adaptive_label = npobj['b_labels']
    
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Benign.csv"
    df.to_csv(path)

In [13]:
def compute_metrics_adv(adv_path, normal_model): 
    
    print("Computing metrics for {} for adv")
    npobj = np.load(adv_path)
    adaptive_image = npobj['a_images']
    adaptive_label = npobj['a_labels']
    
    #attribution robustness
    attribution_gaussian1 = []
    attribution_gaussian2 = []
    attribution_gaussian3 = []
    
    #logit robustness
    logit_gaussian1 = []
    logit_gaussian2 = []
    logit_gaussian3 = []
    
    images, labels = torch.from_numpy(adaptive_image), torch.from_numpy(adaptive_label)
    #images, labels = images.to(device), labels.to(device)
    
    end = len(adaptive_label)
    if end > 1000:
        end = 1000
    
    for i in range(0, end, 2):
        
        images_adv, y_pred_adv = images[i:i+2], labels[i:i+2]
        images_adv, y_pred_adv = images_adv.to(device), y_pred_adv.to(device)
        
        x_logits = normal_model(images_adv)
        gc.collect()
        torch.cuda.empty_cache()
        
        #approach: attribution and logit robustness
        a_batch = quantus.explain(
            model=normal_model, inputs=images_adv, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        gaussian_noisy_images_1 = make_noise(images_adv, y_pred_adv, spread = 0.05)
        gaussian_logits_1 = normal_model(gaussian_noisy_images_1)
        gaussian_noisy_images_2 = make_noise(images_adv, y_pred_adv, spread = 0.10)
        gaussian_logits_2 = normal_model(gaussian_noisy_images_2)
        gaussian_noisy_images_3 = make_noise(images_adv, y_pred_adv, spread = 0.15)
        gaussian_logits_3 = normal_model(gaussian_noisy_images_3)
        
        diff1 = torch.norm(x_logits-gaussian_logits_1,p=1, dim=1) 
        diff2 = torch.norm(x_logits-gaussian_logits_2,p=1, dim=1) 
        diff3 = torch.norm(x_logits-gaussian_logits_3,p=1, dim=1) 
        
        logit_gaussian1.extend(diff1.detach().cpu().numpy())
        logit_gaussian2.extend(diff2.detach().cpu().numpy())
        logit_gaussian3.extend(diff3.detach().cpu().numpy())
        
        
        a_batch_gaussian1 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_1, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian2 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_2, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        a_batch_gaussian3 = quantus.explain(
        model=normal_model, inputs=gaussian_noisy_images_3, targets=y_pred_adv, **{"method:": "IntegratedGradient", "device": device})
        
        for a, b in zip(a_batch, a_batch_gaussian1):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian1.append(c)
            
        for a, b in zip(a_batch, a_batch_gaussian2):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian2.append(c)
        
        for a, b in zip(a_batch, a_batch_gaussian3):
            c = np.linalg.norm(a.flatten()-b.flatten(),ord=1 )
            attribution_gaussian3.append(c)
        
        
    df = pd.DataFrame([
            
            attribution_gaussian1,
            attribution_gaussian2,
            attribution_gaussian3,
            logit_gaussian1,
            logit_gaussian2,
            logit_gaussian3], index = [
            "Gaussian1 attribution", 
            "Gaussian2 attribution", 
            "Gaussian3 attribution", 
            "Gaussian1 logit robusntess",
            "Gaussian2 logit robusntess",
            "Gaussian3 logit robusntess",
                    ])
            
    path = "adaptive_Adv.csv"
    df.to_csv(path)

In [14]:
def compute_TPR(adv1, a, b, adv2, c, d): 
    TN=0
    FN=0
    FP = 0 
    TP=0
    
    for value1, value2 in zip(adv1, adv2): 
        if value1<a or value1>b:
            TP += 1
        else:
            if value2<c or value2>d:
                TP+=1
            else: 
                FN+=1
    
    return (TP/(TP+FN))*100

In [15]:
def compute_FPR(ap2a, k, l, ap2b, m, n): 
    TN=0
    FN=0
    FP=0 
    TP=0
    
    for value6, value7 in zip(ap2a,ap2b):
        if value6<k or value6>l:
            FP +=1
        else:
            if value7<m or value7>n:
                FP +=1

    return (FP/(len(ap2a)))*100

In [16]:
import sklearn
from sklearn.metrics import roc_auc_score

In [19]:
def return_auc(adv_path, model):
    #logitgaussian3


    k=[28,54, 62,76, 86, 96, 106, 126, 150, 200, 400]
    l=[340,340,340,340,340,340,340,340,340,340,340]

    #attrgaussian3
    m=[1800,2200,2700,3100, 3500, 4000, 4500, 5000, 5500, 6000, 8000]
    n=[8500,8500, 8500,8500,8500,8500,8500,8500,8500,8500,8500]

    
    compute_metrics_benign(adv_path, model)
    compute_metrics_adv(adv_path, model)
    df_cifar = pd.read_csv("adaptive_Benign.csv")
    attr_gaussian3 = df_cifar.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3 = df_cifar.iloc[5].values.flatten().tolist()[1:]
        
    fpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        FPR = compute_FPR(logit_gaussian3, t1,t2, attr_gaussian3,t3,t4)
        fpr_results.append(FPR/100)
        
    df_pgd_eps1 = pd.read_csv("adaptive_Adv.csv")
    attr_gaussian3_eps1 = df_pgd_eps1.iloc[2].values.flatten().tolist()[1:]
    logit_gaussian3_eps1 = df_pgd_eps1.iloc[5].values.flatten().tolist()[1:]
    
    tpr_results =[]
    for t1,t2,t3,t4 in zip(k,l,m,n):
        TPR = compute_TPR(logit_gaussian3_eps1, t1,t2, attr_gaussian3_eps1,t3,t4)
        tpr_results.append(TPR/100)
    return(sklearn.metrics.auc(fpr_results, tpr_results), fpr_results, tpr_results)

In [22]:
# FGSM
adv_path1 = 'adv samples/CIFAR/FGSM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/FGSM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/FGSM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/FGSM/0.25098039215686274eps.npz'

In [23]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.87553,
 [0.021, 0.051, 0.096, 0.206, 0.348, 0.549, 0.709, 0.875, 0.965, 0.996, 1.0],
 [0.016,
  0.21899999999999997,
  0.58,
  0.855,
  0.955,
  0.996,
  0.9990000000000001,
  1.0,
  1.0,
  1.0,
  1.0])

In [24]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.975131, [0.018, 0.05, 0.096, 0.209, 0.355, 0.553, 0.704, 0.884, 0.964, 0.995, 1.0], [0.644, 0.9739999999999999, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.988, [0.012, 0.04100000000000001, 0.083, 0.21, 0.331, 0.52, 0.703, 0.8690000000000001, 0.963, 0.996, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.98, [0.02, 0.051, 0.103, 0.214, 0.3390000000000001, 0.534, 0.706, 0.861, 0.963, 0.9940000000000001, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


# PGD

In [25]:
adv_path1 = 'adv samples/CIFAR/PGD/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/PGD/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/PGD/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/PGD/0.25098039215686274eps.npz'

In [26]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.847106,
 [0.013000000000000001,
  0.048,
  0.10100000000000002,
  0.203,
  0.32899999999999996,
  0.515,
  0.6830000000000002,
  0.8539999999999999,
  0.954,
  0.99,
  1.0],
 [0.131,
  0.269,
  0.565,
  0.748,
  0.885,
  0.954,
  0.98,
  0.996,
  0.9990000000000001,
  1.0,
  1.0])

In [27]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.923804, [0.029000000000000005, 0.055, 0.111, 0.225, 0.368, 0.552, 0.7289999999999999, 0.903, 0.978, 0.996, 1.0], [0.368, 0.596, 0.857, 0.9469999999999998, 0.98, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.965324, [0.029000000000000005, 0.05, 0.109, 0.21099999999999997, 0.348, 0.531, 0.709, 0.882, 0.966, 0.992, 1.0], [0.6919999999999998, 0.9469999999999998, 0.996, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9799650000000001, [0.02, 0.055, 0.106, 0.22, 0.337, 0.537, 0.711, 0.883, 0.966, 0.997, 1.0], [0.998, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


# BIM

In [28]:
adv_path1 = 'adv samples/CIFAR/BIM/0.03137254901960784eps.npz'
adv_path2 = 'adv samples/CIFAR/BIM/0.06274509803921569eps.npz'
adv_path3 = 'adv samples/CIFAR/BIM/0.12549019607843137eps.npz'
adv_path4 = 'adv samples/CIFAR/BIM/0.25098039215686274eps.npz'

In [29]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.765647,
 [0.014000000000000002,
  0.05800000000000001,
  0.106,
  0.204,
  0.342,
  0.532,
  0.708,
  0.872,
  0.968,
  0.992,
  1.0],
 [0.04,
  0.146,
  0.391,
  0.613,
  0.773,
  0.8859999999999999,
  0.95,
  0.978,
  0.9940000000000001,
  0.9990000000000001,
  1.0])

In [30]:
print(return_auc(adv_path2, normal_model))
print('----')
print(return_auc(adv_path3, normal_model))
print('----')
print(return_auc(adv_path4, normal_model))

Computing metrics for {} for benign
Computing metrics for {} for adv
(0.8440654999999999, [0.014000000000000002, 0.04100000000000001, 0.102, 0.207, 0.32899999999999996, 0.517, 0.709, 0.885, 0.965, 0.996, 1.0], [0.103, 0.255, 0.543, 0.764, 0.879, 0.951, 0.983, 0.992, 0.998, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.9255890000000001, [0.019, 0.061, 0.111, 0.22699999999999998, 0.353, 0.525, 0.7070000000000001, 0.878, 0.973, 0.997, 1.0], [0.218, 0.518, 0.875, 0.9620000000000001, 0.99, 0.998, 1.0, 1.0, 1.0, 1.0, 1.0])
----
Computing metrics for {} for benign
Computing metrics for {} for adv
(0.975921, [0.017, 0.039, 0.095, 0.18600000000000003, 0.327, 0.519, 0.687, 0.862, 0.958, 0.9940000000000001, 1.0], [0.61, 0.939, 0.996, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])


# CW

In [31]:
adv_path1 = 'adv samples/CIFAR/CW/cwCIFAR.npz'

In [32]:
auc, tpr, fpr = return_auc(adv_path1, normal_model)
auc, tpr, fpr

Computing metrics for {} for benign
Computing metrics for {} for adv


(0.9758230000000001,
 [0.019,
  0.057,
  0.106,
  0.22400000000000003,
  0.332,
  0.527,
  0.699,
  0.865,
  0.961,
  0.9990000000000001,
  1.0],
 [0.806, 0.976, 0.996, 0.9990000000000001, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])